In [ ]:
import pika

# Establish a connection to RabbitMQ
rabbitmq_host = "localhost"
rabbitmq_port = 5672
username = "guest"
password = "guest"
queue_name = "queue"


def callback(ch, method, properties, body) -> None:
    """
    Callback function to process incoming messages from RabbitMQ.

    Args:
        ch: RabbitMQ channel
        method: RabbitMQ method
        properties: RabbitMQ properties
        body: Message body

    Returns:
        None
    """
    print(f"Received {body}")
    
# Create a connection to the RabbitMQ server with credentials
credentials = pika.PlainCredentials(username, password)
connection = pika.BlockingConnection(
    pika.ConnectionParameters(
        host=rabbitmq_host,
        port=rabbitmq_port,
        credentials=credentials,
        heartbeat=0,
    )
)
channel = connection.channel()

# Declare the queue (this is idempotent, it won't create the queue if it already exists)
channel.queue_declare(queue=queue_name, durable=True)

# Set up the consumer
channel.basic_consume(queue=queue_name, on_message_callback=callback)

# Start consuming messages
print("Waiting for a single message. To exit, press CTRL+C")
channel.start_consuming()